In [155]:
import json
import numpy as np

In [115]:
#每日数据
#由于这里的数据处理吴雨杭应该有现成的算法，这里应该等他的算法然后直接生成一个新的细粒度的序列
d=dict()
for order in range(1,4): 
    with open('RefinedDay'+str(order)+'Dic.json','r') as load_f:
         d[order] = json.load(load_f)

In [116]:
#提取序列特征
feat=(0,dict(),dict(),dict())

In [119]:
#求停留点序列
def parsevec(vec,ts):
    res=[]
    delta=0
    leng=len(vec)
    for i in range(leng):
        if(i+1<leng and vec[i+1][0]-vec[i][0]>ts):
            res.append(vec[i][1])
    return res
#求序列编辑距离
def edit_distance(word1, word2):
    """opensource algs"""    
    len1 = len(word1);
    len2 = len(word2);
    dp = np.zeros((len1 + 1,len2 + 1))
    for i in range(len1 + 1):
        dp[i][0] = i;     
    for j in range(len2 + 1):
        dp[0][j] = j;
     
    for i in range(1, len1 + 1):
        for j in range(1, len2 + 1):
            delta = 0 if word1[i-1] == word2[j-1] else 1
            dp[i][j] = min(dp[i - 1][j - 1] + delta, min(dp[i-1][j] + 1, dp[i][j - 1] + 1))
    return dp[len1][len2]

In [120]:
#计算停留点序列
ids = set()
for day in d:
    for k in d[day]:
        ids.add(k)
        feat[day][k]=parsevec(d[day][k],5)

In [129]:
#所有可能的位置
pos=set()
for arr in feat[1:4]:
    for k in arr:
        for i in arr[k]:
            pos.add(i)

In [140]:
#解码器/编码器
decoder=dict()
encoder=dict()
i=ord('a')
for k in pos:
    encoder[k]=chr(i)
    decoder[chr(i)]=k
    i=i+1

In [143]:
def encode(ls):
    st=''
    for i in ls:
        st+=encoder[i]
    return st

In [144]:
#数据重塑
res=dict()
for i in ids:
    res[i]=[0,0,0]
for i in ids:
    for k in range(0,3):
        if i in feat[k+1]:
            res[i][k]=encode(feat[k+1][i])
        else:
            res[i][k]=''

In [156]:
js= json.dumps(res)

In [159]:
en= json.dumps(encoder)

In [160]:
de=json.dumps(decoder)

In [163]:
f = open("feature.json", "w")
f.write(js)
f.close()

In [164]:
f = open("en.json", "w")
f.write(en)
f.close()

In [165]:
f = open("de.json", "w")
f.write(de)
f.close()